In [1]:

import ibis
from ibis import _
from cng.utils import *
from cng.h3 import * 
import os
con = ibis.duckdb.connect("local.db", extensions = ["spatial", "h3"])
install_h3()

os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

# nwi = con.read_parquet("s3://us-west-2.opendata.source.coop/giswqs/nwi/wetlands/**")

def geom_to_cell (df, zoom = 8):
    con = df.get_backend() # ibis >= 10.0

    # First make sure we are using multipolygons everywhere and not a mix
    cases = ibis.cases(
        (df.geom.geometry_type() == 'POLYGON' , ST_Multi(df.geom)),
        else_=df.geom,
    )
    
    df = df.mutate(geom = cases)
    sql = ibis.to_sql(df)
    expr = f'''
        WITH t1 AS (
        SELECT ATTRIBUTE, GLOBALID, WETLAND_TYPE, UNNEST(ST_Dump(ST_GeomFromWKB(geom))).geom AS geom 
        FROM ({sql})
        ) 
        SELECT *, h3_polygon_wkt_to_cells_string(geom, {zoom}) AS h3id  FROM t1
    '''

    out = con.sql(expr)
    return out



In [ ]:
import minio
mc = minio.Minio("s3.amazonaws.com", "", "", region="us-west-2")
obj = mc.list_objects("us-west-2.opendata.source.coop", prefix = "giswqs/nwi/wetlands", recursive = True)

for o in obj:
    nwi = con.read_parquet("s3://us-west-2.opendata.source.coop/" + o.object_name)
    part_name = o.object_name.split('/')[-1]
    state = part_name.split('_')[0]
    print(state)
    if state not "AK":
        (
            geom_to_cell(nwi.rename(geom = "geometry"), zoom=8)
            .mutate(h8 = _.h3id.unnest())
            .to_parquet(f"s3://public-wetlands/nwi/hex/state={state}/data_0.parquet")
        )
    



AK
